Loading required libraries

In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows",None,"display.max_columns", None)
import warnings
warnings.filterwarnings('always') 

In [2]:
!pip install scikit-learn
import numpy as np
import matplotlib.pyplot as plt  # To visualize
import pandas as pd  # To read data

sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x29d67823ac0>
c:\users\study\onedrive\desktop\du\business_intelligence\labs\venv\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
c:\users\study\onedrive\desktop\du\business_intelligence\labs\venv\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
c:\users\study\onedrive\desktop\du\business_intelligence\labs\venv\lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)


Loading dataset
Since the dataset is huge, we sample only 1 % of it in our study. 
We then cleanse our dataset acccording to our model    

In [3]:
hmdata = pd.read_csv('C:/Users/Study/OneDrive/Desktop/DU/Business_Intelligence/Home_assignment_BI/dataset_hm.csv')
hmdata.head(10) 
hmtrain = hmdata.sample(frac=0.1, random_state=100)
hmtrain.shape



(7986, 14)

In [10]:
# filter price
hmtrain = hmtrain[(hmtrain['Price']>0)]
hmtrain.shape
hmtrain[(hmtrain['Price']<0)].shape

(0, 17)

In [11]:
# Changing date format 
hmtrain['t_date'] = pd.to_datetime(hmtrain['t_date'],infer_datetime_format=True)
hmtrain['t_date'].dt.strftime('%d/%m/%Y')
hmtrain['t_month'] = pd.to_datetime(hmtrain['t_date']).dt.month
hmtrain['t_month'].head(6)
hmtrain.head(6)

,t_date,product_type_no,product_type_name,product_group_no,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,index_group_no,index_group_name,Price,t_month,idxgrp_idx_prdtyp,year
65748,2019-08-20,252,Sweater,3,Garment Upper body,1010008,Front print,73,Dark Blue,4,Dark,4,Baby/Children,0.012315,8,Baby/Children_Sweater,2019
48841,2019-08-21,252,Sweater,3,Garment Upper body,1010001,All over pattern,7,Grey,2,Medium Dusty,3,Menswear,0.033728,8,Menswear_Sweater,2019
58116,2019-09-19,255,T-shirt,3,Garment Upper body,1010014,Placement print,42,Red,5,Bright,4,Baby/Children,0.027390,9,Baby/Children_T-shirt,2019
62171,2019-10-18,272,Trousers,2,Garment Lower body,1010016,Solid,73,Dark Blue,4,Dark,4,Baby/Children,0.048582,10,Baby/Children_Trousers,2019
60294,2019-02-02,308,Hoodie,3,Garment Upper body,1010016,Solid,9,Black,4,Dark,2,Divided,0.028780,2,Divided_Hoodie,2019
7994,2019-07-16,272,Trousers,2,Garment Lower body,1010010,Melange,10,White,3,Light,4,Baby/Children,0.040482,7,Baby/Children_Trousers,2019


In [6]:
hmtrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7471 entries, 65748 to 25965
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   t_date                       7471 non-null   datetime64[ns]
 1   product_type_no              7471 non-null   int64         
 2   product_type_name            7471 non-null   object        
 3   product_group_no             7471 non-null   int64         
 4   product_group_name           7471 non-null   object        
 5   graphical_appearance_no      7471 non-null   int64         
 6   graphical_appearance_name    7471 non-null   object        
 7   colour_group_code            7471 non-null   int64         
 8   colour_group_name            7471 non-null   object        
 9   perceived_colour_value_id    7471 non-null   int64         
 10  perceived_colour_value_name  7471 non-null   object        
 11  index_group_no               7471 non-




We create a new dataframe hmtraingrp1 to keep only the columns we require to apply PCA

In [12]:
hmtrain['idxgrp_idx_prdtyp'] = hmtrain['index_group_name'] + '_' + hmtrain['product_type_name']
hmtrain['idxgrp_idx_prdtyp'].head(6)
hmtrain['year'] = hmtrain['t_date'].dt.year
hmtrain['t_month'] = hmtrain['t_date'].dt.month
hmtraingrp1 = hmtrain.groupby(['idxgrp_idx_prdtyp'])[['Price']].sum().reset_index()
hmtraingrp2 = hmtrain.groupby(['idxgrp_idx_prdtyp','t_month','year'])[['Price']].sum().reset_index()
hmtraingrp2 = pd.merge(hmtraingrp2, hmtraingrp1, on='idxgrp_idx_prdtyp', how='left')
hmtraingrp2['monthsales/ttl-sales'] = hmtraingrp2['Price_x'] / hmtraingrp2['Price_y'] * 100
hmtraingrp2['ym_date'] = hmtraingrp2['year'].astype(str) + '-' + hmtraingrp2['t_month'].astype(str) + '-1'
hmtraingrp2['ym_date'] = pd.to_datetime(hmtraingrp2['ym_date'])
hmtraingrp2.head(5)


,idxgrp_idx_prdtyp,t_month,year,Price_x,Price_y,monthsales/ttl-sales,ym_date
0,Baby/Children_Blazer,4,2019,0.076843,0.233459,32.914943,2019-04-01
1,Baby/Children_Blazer,6,2019,0.068315,0.233459,29.262149,2019-06-01
2,Baby/Children_Blazer,7,2019,0.015414,0.233459,6.602254,2019-07-01
3,Baby/Children_Blazer,9,2019,0.002060,0.233459,0.882184,2019-09-01
4,Baby/Children_Blazer,10,2019,0.049804,0.233459,21.332986,2019-10-01


We create a pivot table from the hmtraingrp2 which is then inputted in the PCA model 

In [13]:
hmtrain_df = hmtraingrp2 #creating backup of hmtraingrp2, just in case
hmtraingrp2 = pd.pivot_table(hmtraingrp2, index='ym_date', columns='idxgrp_idx_prdtyp', values='monthsales/ttl-sales').reset_index().fillna(0)
display(hmtraingrp2.head())


idxgrp_idx_prdtyp,ym_date,Baby/Children_Blazer,Baby/Children_Blouse,Baby/Children_Bodysuit,Baby/Children_Boots,Baby/Children_Cardigan,Baby/Children_Coat,Baby/Children_Costumes,Baby/Children_Dress,Baby/Children_Dungarees,Baby/Children_Flat shoe,Baby/Children_Flip flop,Baby/Children_Hoodie,Baby/Children_Jacket,Baby/Children_Jumpsuit/Playsuit,Baby/Children_Leggings/Tights,Baby/Children_Polo shirt,Baby/Children_Pyjama bottom,Baby/Children_Sandals,Baby/Children_Shirt,Baby/Children_Shorts,Baby/Children_Skirt,Baby/Children_Sneakers,Baby/Children_Sweater,Baby/Children_Swimsuit,Baby/Children_T-shirt,Baby/Children_Top,Baby/Children_Trousers,Baby/Children_Vest top,Divided_Bikini top,Divided_Blazer,Divided_Blouse,Divided_Bodysuit,Divided_Boots,Divided_Cardigan,Divided_Coat,Divided_Dress,Divided_Dungarees,Divided_Flat shoe,Divided_Flip flop,Divided_Hoodie,Divided_Jacket,Divided_Jumpsuit/Playsuit,Divided_Leggings/Tights,Divided_Pumps,Divided_Sandals,Divided_Shirt,Divided_Shorts,Divided_Skirt,Divided_Sneakers,Divided_Sweater,Divided_Swimsuit,Divided_T-shirt,Divided_Top,Divided_Trousers,Divided_Vest top,Divided_Wedge,Ladieswear_Bikini top,Ladieswear_Blazer,Ladieswear_Blouse,Ladieswear_Bodysuit,Ladieswear_Boots,Ladieswear_Cardigan,Ladieswear_Coat,Ladieswear_Costumes,Ladieswear_Dress,Ladieswear_Flat shoe,Ladieswear_Flip flop,Ladieswear_Hoodie,Ladieswear_Jacket,Ladieswear_Jumpsuit/Playsuit,Ladieswear_Leggings/Tights,Ladieswear_Polo shirt,Ladieswear_Pumps,Ladieswear_Pyjama bottom,Ladieswear_Sandals,Ladieswear_Shirt,Ladieswear_Shorts,Ladieswear_Skirt,Ladieswear_Sneakers,Ladieswear_Sweater,Ladieswear_Swimsuit,Ladieswear_T-shirt,Ladieswear_Top,Ladieswear_Trousers,Ladieswear_Vest top,Ladieswear_Wedge,Mama_Blouse,Mama_Dress,Mama_Dungarees,Mama_Hoodie,Mama_Jacket,Mama_Leggings/Tights,Mama_Shirt,Mama_Shorts,Mama_Skirt,Mama_Sweater,Mama_T-shirt,Mama_Top,Mama_Trousers,Mama_Vest top,Menswear_Blazer,Menswear_Boots,Menswear_Cardigan,Menswear_Coat,Menswear_Flip flop,Menswear_Hoodie,Menswear_Jacket,Menswear_Polo shirt,Menswear_Pyjama bottom,Menswear_Sandals,Menswear_Shirt,Menswear_Shorts,Menswear_Sneakers,Menswear_Sweater,Menswear_T-shirt,Menswear_Top,Menswear_Trousers,Menswear_Vest top,Sport_Bikini top,Sport_Hoodie,Sport_Jacket,Sport_Leggings/Tights,Sport_Shorts,Sport_Skirt,Sport_Sweater,Sport_T-shirt,Sport_Top,Sport_Trousers,Sport_Vest top
0,2019-01-01,0.000000,6.303199,12.417352,19.045830,3.588803,0.000000,0.00000,9.764627,19.506538,0.0,0.0,6.988347,5.195795,8.956111,7.493526,0.000000,0.000000,6.124670,4.258904,11.975772,3.382258,13.667443,7.279429,11.941819,8.099974,4.501385,6.616941,3.316538,0.000000,14.512992,8.909308,0.000000,31.93357,8.049003,13.431159,9.062424,5.328354,0.0,0.0,4.248393,7.006242,0.000000,0.000000,0.000000,0.000000,7.328526,11.348631,12.624216,0.000000,13.711682,0.0,12.909023,17.972537,10.987286,0.897703,0.0,5.424441,16.118394,10.288805,22.354497,12.135786,3.625102,3.439388,0.0,8.496651,0.000000,0.0,4.909607,6.490705,6.800765,0.000000,0.0,24.748785,7.743122,5.093872,3.323887,5.228762,5.776455,9.835185,7.576345,5.009910,7.471761,7.877292,6.690630,8.894658,14.776245,11.445495,8.167600,0.0,0.0,0.0,0.0,11.644956,6.888790,0.000000,0.000000,13.397633,18.543087,6.225377,22.990930,2.970256,26.994173,0.000000,8.701523,0.000000,19.283668,4.418523,14.978031,11.488860,0.000000,6.111974,8.272117,5.824312,5.285711,6.105708,11.942273,10.987265,19.048139,0.0,7.208807,15.685133,9.311371,2.128620,0.0,5.295761,4.882291,6.999806,6.061271,17.823269
1,2019-02-01,0.000000,8.241257,7.481470,7.569144,18.786410,0.000000,0.00000,11.025010,3.006534,0.0,0.0,10.264796,3.398480,12.287756,12.831751,0.000000,81.879451,8.176638,7.492396,9.006790,7.279274,8.225751,11.036523,12.580496,6.400308,5.130695,5.731506,12.867919,36.336643,9.900682,11.250020,0.000000,0.00000,5.778156,0.000000,8.558287,0.000000,0.0,0.0,18.852408,10.901827,0.000000,0.000000,0.000000,9.905465,7.646983,14.318266,4.903866,8.112215,7.040387,0.0,10.845918,6.330269,4.856348,6.839065,0.0,7.135436,5.097194,

We analyse the monthly trends of the products using below graph

In [25]:
import plotly.express as px
fig = px.line(hmtraingrp2, x='ym_date', y=['Ladieswear_Jacket',
                                      'Divided_Jacket',
                                      'Menswear_Jacket',
                                      'Ladieswear_Sweater',
                                      'Divided_Sweater',
                                      'Menswear_Sweater'], title="MothlySales / TTL sales")
fig.show()

In [ ]:
'Menswear_Sneakers',
                                      'Sport_T-shirt',
                                      'Sport_Hoodie','Ladieswear_Top','Menswear_Boots' 

We now apply PCA and then plot PC1 vs PC2 to see how products are classified according to the seasonality.

In [26]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


In [57]:
feat_cols = [col for col in hmtraingrp2.columns if col != 'ym_date']
df_pca = StandardScaler().fit_transform(hmtraingrp2[feat_cols])
model_pca = PCA(n_components=5)
model_pca.fit(df_pca)
feature = model_pca.transform(df_pca)

df_eigen = model_pca.components_.T
df_eigen = pd.DataFrame(df_eigen,
                        index=None,
                        columns=['PC1','PC2','PC3','PC4','PC5'])
df_eigen['idxgrp_idx_prdtyp'] = feat_cols
df_eigen.head(5)
df_eigen = pd.merge(
     df_eigen,
     hmtraingrp2.corr()[['Ladieswear_Jacket']].reset_index().rename(columns={'Ladieswear_Jacket': 'Summer_sales_indicator'}),
     on='idxgrp_idx_prdtyp',
     how='left'
)


px.scatter(df_eigen, x='PC1', y='PC2', hover_name='idxgrp_idx_prdtyp', color='Summer_sales_indicator')



Index(['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'idxgrp_idx_prdtyp',
       'Summer_sales_indicator'],
      dtype='object')

We see that the summer products like skirts, wedges, bikini tops etc. are grouped together, represented by the lighter dots. Likewise, winter products like jackets, sweater etc. are grouped together and represented by darker dots.

In [ ]:
hmtrain[product_type_name].values()